In [ ]:
import lsst.afw.table
import lsst.afw.image
import lsst.afw.math
import lsst.meas.algorithms
import lsst.meas.base
import lsst.meas.deblender
import numpy as np
import astropy.io.fits as fits
import matplotlib.pyplot as plt
import descwl
%matplotlib inline
import astropy.table
import scipy.spatial
import scipy.ndimage
from lsst.sims.GalSimInterface.wcsUtils import tanSipWcsFromDetector
from lsst.sims.GalSimInterface import LSSTCameraWrapper
from lsst.sims.utils import ObservationMetaData
schema = lsst.afw.table.SourceTable.makeMinimalSchema()
config1 = lsst.meas.algorithms.SourceDetectionConfig()
config1.tempLocalBackground.binSize=8
config1.minPixels=1
config1.thresholdValue=12
detect = lsst.meas.algorithms.SourceDetectionTask(schema=schema, config=config1)
deblend = lsst.meas.deblender.SourceDeblendTask(schema=schema)
config1 = lsst.meas.base.SingleFrameMeasurementConfig()
## We can uncomment this once we add the HSM extension to the common stack install
#config1.plugins.names.add('ext_shapeHSM_HsmShapeBj')
#config1.plugins.names.add('ext_shapeHSM_HsmShapeLinear')
#config1.plugins.names.add('ext_shapeHSM_HsmShapeKsb')
#config1.plugins.names.add('ext_shapeHSM_HsmShapeRegauss')
#config1.plugins.names.add('ext_shapeHSM_HsmSourceMoments')
#config1.plugins.names.add('ext_shapeHSM_HsmPsfMoments')
measure = lsst.meas.base.SingleFrameMeasurementTask(schema=schema, config=config1)
camera_wrapper = LSSTCameraWrapper()
obs = ObservationMetaData(pointingRA=0, pointingDec=0,
                          boundType='circle', boundLength=2.0,
                          mjd=52000.0, rotSkyPos=0,
                          bandpassName='i')

In [ ]:
def process(input_path, output_path=None,seed=123):
    LSST_i = descwl.output.Reader(input_path).results
    LSST_i.add_noise(noise_seed=seed)
    image = lsst.afw.image.ImageF(LSST_i.survey.image.array)
    variance_array = np.sqrt(LSST_i.survey.image.array)
    variance = lsst.afw.image.ImageF(variance_array)
    masked_image = lsst.afw.image.MaskedImageF(image, None, variance)
    psf_array = LSST_i.survey.psf_image.array
    psf_array = psf_array.astype(np.float64)
    psf_new = scipy.ndimage.zoom(psf_array,zoom=43/76.)
    im = lsst.afw.image.ImageD(psf_new)
    fkernel = lsst.afw.math.FixedKernel(im)
    psf = lsst.meas.algorithms.KernelPsf(fkernel)
    exposure = lsst.afw.image.ExposureF(masked_image)
    exposure.setPsf(psf)
    wcs_in = tanSipWcsFromDetector('R:2,2 S:1,1',camera_wrapper,obs,2000)
    #wcs = lsst.afw.image.Wcs(crval,crpix,cd)
    exposure.setWcs(wcs_in)
    table = lsst.afw.table.SourceTable.make(schema)  # this is really just a factory for records, not a table
    detect_result = detect.run(table, exposure)
    catalog = detect_result.sources   # this is the actual catalog, but most of it's still empty
    deblend.run(exposure, catalog)
    measure.run(catalog, exposure)
    catalog = catalog.copy(deep=True)
    if output_path is not None:
        catalog.writeFits(output_path)
    return catalog

In [ ]:
catalog = process('LSST_i.fits')

In [ ]:
catalog.schema